<a href="https://www.kaggle.com/code/hatemamine/eval-retrived-and-reranked-list-with-pyserini?scriptVersionId=222356795" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# pyserini love java (install java)
!sudo apt-get update
!sudo apt-get install openjdk-21-jre -y
!java -version
!sudo  update-java-alternatives --list
!export JAVA_HOME='/usr/lib/jvm/java-1.21.0-openjdk-amd64'
import os
# linux
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.21.0-openjdk-amd64"
# installing pyserini
#https://github.com/castorini/pyserini
!python --version
!pip install faiss-gpu -q
!pip install pyserini -q
# install numpy==1.26.4 to fix package conflit 
import numpy 
numpy.__version__
#!pip show numpy
!pip install --upgrade numpy==1.26.4

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:7 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [33.9 kB]
Get:8 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3589 kB]
Get:9 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1635 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [34.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [4526 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4699 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/uni

In [2]:
# 100 bm25 mr.Tydi
!python -m pyserini.search.lucene \
  --threads 16 --batch-size 128 \
  --language ar \
  --topics mrtydi-v1.1-arabic-test \
  --index mrtydi-v1.1-ar \
  --output run.mrtydi.bm25.ar.test.txt --bm25 --hits 1000
!python -m pyserini.eval.trec_eval \
  -c -m recip_rank.100 -m recall.10,100 mrtydi-v1.1-arabic-test \
  run.mrtydi.bm25.ar.test.txt
!python -m pyserini.eval.trec_eval \
-c -m recip_rank.100 -m recall.10,1000 mrtydi-v1.1-arabic-test \
  run.mrtydi.bm25.ar.test.txt

2025-02-13 13:06:26,454 INFO  [main] topicreader.TopicReader (TopicReader.java:229) - Downloading topics from https://raw.githubusercontent.com/castorini/anserini-tools/master/topics-and-qrels/topics.mrtydi-v1.1-ar.test.txt.gz
lucene-inverted.mrtydi-v1.1-ar.20220928.b5ecc5.tar.gz: 1.09GB [00:14, 81.4MB/s] 
Feb 13, 2025 1:07:17 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false
Running mrtydi-v1.1-arabic-test topics, saving to run.mrtydi.bm25.ar.test.txt...
100%|███████████████████████████████████████| 1081/1081 [00:46<00:00, 23.27it/s]
recip_rank            	all	0.3687
recall_10             	all	0.5631
recall_100            	all	0.7928
recip_rank            	all	0.3687
recall_10             	all	0.5631
recall_1000           	all	0.9055


#zero-shot evaluation
#mr.tydi
https://castorini.github.io/pyserini/2cr/mrtydi.html
https://github.com/castorini/mr.tydi

In [3]:
from huggingface_hub import hf_hub_download
organization_dataset_id= "hatemestinbejaia/ExperimentDATA_knowledge_distillation_vs_fine_tuning"
#zero-shot evaluation reranked bm25-100 mr.Tydi
hf_hub_download(repo_id=organization_dataset_id, filename="qrels.test.mrtydi.txt", local_dir="./", repo_type="dataset")
ranked_list_mrTydi=["mrtydi_reranked/mmarco-Arabic-mMiniLML-cross-encoder-KD-v1.tsvrun.mrtydi.bm25.ar.test.txt", "mrtydi_reranked/mmarco-Arabic-mMiniLML-cross-encoder-NoKD-v1.tsvrun.mrtydi.bm25.ar.test.txt", "mrtydi_reranked/mmarco-Arabic-AraDPR-cross-encoder-KD-v1_run.mrtydi.bm25.ar.test.txt", "mrtydi_reranked/mmarco-Arabic-AraDPR-cross-encoder-NoKD-v1_run.mrtydi.bm25.ar.test.txt", "mrtydi_reranked/mmarco-Arabic-AraElectra-cross-encoder-KD-v1_run.mrtydi.bm25.ar.test.txt", "mrtydi_reranked/mmarco-Arabic-AraElectra-cross-encoder-NoKD-v1_run.mrtydi.bm25.ar.test.txt" ]
for i in range(len(ranked_list_mrTydi)):
    hf_hub_download(repo_id=organization_dataset_id, filename=ranked_list_mrTydi[i], local_dir="./", repo_type="dataset")
for i in ranked_list_mrTydi:
        print("*****************************************")
        print(i)
        print("*****************************************")
        !python -m pyserini.eval.trec_eval  -c -m recip_rank.100 -m recall.100\
            qrels.test.mrtydi.txt \
            {i}
        #break

qrels.test.mrtydi.txt:   0%|          | 0.00/24.9k [00:00<?, ?B/s]

(…)der-KD-v1.tsvrun.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

(…)r-NoKD-v1.tsvrun.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

(…)ncoder-KD-v1_run.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

(…)oder-NoKD-v1_run.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

(…)ncoder-KD-v1_run.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

(…)oder-NoKD-v1_run.mrtydi.bm25.ar.test.txt:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

*****************************************
mrtydi_reranked/mmarco-Arabic-mMiniLML-cross-encoder-KD-v1.tsvrun.mrtydi.bm25.ar.test.txt
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.5149
recall_100            	all	0.7928
*****************************************
mrtydi_reranked/mmarco-Arabic-mMiniLML-cross-encoder-NoKD-v1.tsvrun.mrtydi.bm25.ar.test.txt
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.4927
recall_100            	all	0.7928
*****************************************
mrtydi_reranked/mmarco-Arabic-AraDPR-cross-encoder-KD-v1_run.mrtydi.bm25.ar.test.txt
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.5664
recall_100            	all	0.7928
*****************************************
mrtydi_reranked/mmarco-Arabic-AraDPR-cross-encoder-NoKD-v1_run.mrtydi.bm25.ar.test.txt
****************

In [4]:
from huggingface_hub import hf_hub_download
#zero-shot evaluation first stage mr.Tydi
hf_hub_download(repo_id=organization_dataset_id, filename="qrels.test.mrtydi.txt", local_dir="./", repo_type="dataset")
ranked_list_mrTydi=["FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-KDaradpr.tsv", "FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-NOKDaradpr.tsv", "FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-mmarco-Arabic-AraElectra-bi-encoder-KD-v1.tsv", "FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1.tsv", "FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-mmarco-Arabic-mMiniLML-bi-encoder-KD-v1.tsv", "FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1.tsv", "FirstStagecombine.tsv", "FirstStagecombine1000.tsv", "mmarco-Arabic-AraElectra-cross-encoder-KD-v1_FirstStagecombine1000.tsv", "mmarco-Arabic-AraElectra-cross-encoder-KD-v1_run.mrtydi.bm25.ar.test1000.txt" ]
for i in range(len(ranked_list_mrTydi)):
    hf_hub_download(repo_id=organization_dataset_id, filename=ranked_list_mrTydi[i], local_dir="./", repo_type="dataset")
for i in ranked_list_mrTydi:
        print("*****************************************")
        print(i)
        print("*****************************************")
        !python -m pyserini.eval.trec_eval   -m  recip_rank.100 -m recall.100 \
            qrels.test.mrtydi.txt \
            {i}
        #break

(…)collection-arabic-embedding-KDaradpr.tsv:   0%|          | 0.00/21.2M [00:00<?, ?B/s]

(…)llection-arabic-embedding-NOKDaradpr.tsv:   0%|          | 0.00/2.02M [00:00<?, ?B/s]

(…)o-Arabic-AraElectra-bi-encoder-KD-v1.tsv:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

(…)Arabic-AraElectra-bi-encoder-NoKD-v1.tsv:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

(…)rco-Arabic-mMiniLML-bi-encoder-KD-v1.tsv:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

...

FirstStagecombine.tsv:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

FirstStagecombine1000.tsv:   0%|          | 0.00/21.2M [00:00<?, ?B/s]

(…)-encoder-KD-v1_FirstStagecombine1000.tsv:   0%|          | 0.00/21.2M [00:00<?, ?B/s]

(…)er-KD-v1_run.mrtydi.bm25.ar.test1000.txt:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

*****************************************
FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-KDaradpr.tsv
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.3046
recall_100            	all	0.6818
*****************************************
FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-NOKDaradpr.tsv
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.2348
recall_100            	all	0.5245
*****************************************
FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-mmarco-Arabic-AraElectra-bi-encoder-KD-v1.tsv
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.2110
recall_100            	all	0.5450
*****************************************
FirstStage_mrTydi/FirstStagemrTydi-collection-arabic-embedding-mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1.

In [5]:
# evaluate first stage mmarco
qerl_file ="qrels.msmarco-passage.dev-subset.txt"
ranked_list=["FirstStage_mmarco/FirstStage-mmarco-Arabic-mMiniLML-bi-encoder-KD-v1.tsv", "FirstStage_mmarco/FirstStage-mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1.tsv", "FirstStage_mmarco/FirstStage-mmarco-Arabic-AraDPR-bi-encoder-KD-v1.tsv", "FirstStage_mmarco/FirstStage-mmarco-Arabic-AraDPR-bi-encoder-NoKD-v1.tsv", "FirstStage_mmarco/FirstStage-mmarco-Arabic-AraElectra-bi-encoder-KD-v1.tsv", "FirstStage_mmarco/FirstStage-mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1.tsv", "FirstStageArabicmmarcopassageAraDPRetmMiniLM.dev.small.tsv" ]
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id=organization_dataset_id, filename=qerl_file, local_dir="./", repo_type="dataset")
for i in range(len(ranked_list)):
    hf_hub_download(repo_id=organization_dataset_id, filename=ranked_list[i], local_dir="./", repo_type="dataset")
for i in ranked_list:
        print("*****************************************")
        print(i)
        print("*****************************************")
        #!python -m pyserini.eval.msmarco_passage_eval \ # to get only MRR@10 
        !python -m pyserini.eval.trec_eval  -c -m recip_rank.10 -m recall.1000\
            qrels.msmarco-passage.dev-subset.txt \
            {i}
        #break

qrels.msmarco-passage.dev-subset.txt:   0%|          | 0.00/143k [00:00<?, ?B/s]

(…)rco-Arabic-mMiniLML-bi-encoder-KD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)o-Arabic-mMiniLML-bi-encoder-NoKD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)marco-Arabic-AraDPR-bi-encoder-KD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)rco-Arabic-AraDPR-bi-encoder-NoKD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)o-Arabic-AraElectra-bi-encoder-KD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)Arabic-AraElectra-bi-encoder-NoKD-v1.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

(…)arcopassageAraDPRetmMiniLM.dev.small.tsv:   0%|          | 0.00/133M [00:00<?, ?B/s]

*****************************************
FirstStage_mmarco/FirstStage-mmarco-Arabic-mMiniLML-bi-encoder-KD-v1.tsv
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.1839
recall_1000           	all	0.7771
*****************************************
FirstStage_mmarco/FirstStage-mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1.tsv
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.1420
recall_1000           	all	0.6365
*****************************************
FirstStage_mmarco/FirstStage-mmarco-Arabic-AraDPR-bi-encoder-KD-v1.tsv
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.1748
recall_1000           	all	0.7617
*****************************************
FirstStage_mmarco/FirstStage-mmarco-Arabic-AraDPR-bi-encoder-NoKD-v1.tsv
*****************************************
msmarco run detected. Converting to 

In [6]:
# evaluate second stage Arabic mMARCO reranked 1000bm25
qerl_file ="qrels.msmarco-passage.dev-subset.txt"
ranked_list=["mmarco_run1000bm25Reranked/mmarco-Arabic-mMiniLML-cross-encoder-KD-v1.tsv", "mmarco_run1000bm25Reranked/mmarco-Arabic-mMiniLML-cross-encoder-NoKD-v1.tsv", "mmarco_run1000bm25Reranked/mmarco-Arabic-AraDPR-cross-encoder-KD-v1.tsv", "mmarco_run1000bm25Reranked/mmarco-Arabic-AraDPR-cross-encoder-NoKD-v1.tsv", "mmarco_run1000bm25Reranked/mmarco-Arabic-AraElectra-cross-encoder-KD-v1.tsv", "mmarco_run1000bm25Reranked/mmarco-Arabic-AraElectra-cross-encoder-NoKD-v1.tsv" ]
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id=organization_dataset_id, filename=qerl_file, local_dir="./", repo_type="dataset")
for i in range(len(ranked_list)):
    hf_hub_download(repo_id=organization_dataset_id, filename=ranked_list[i], local_dir="./", repo_type="dataset")
for i in ranked_list:
        print("*****************************************")
        print(i)
        print("*****************************************")
        #!python -m pyserini.eval.msmarco_passage_eval \ # to get only MRR@10 
        !python -m pyserini.eval.trec_eval  -c  -m recip_rank.10 -m recall.1000\
            qrels.msmarco-passage.dev-subset.txt \
            {i}
        #break

(…)-Arabic-mMiniLML-cross-encoder-KD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

(…)rabic-mMiniLML-cross-encoder-NoKD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

(…)co-Arabic-AraDPR-cross-encoder-KD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

(…)-Arabic-AraDPR-cross-encoder-NoKD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

(…)rabic-AraElectra-cross-encoder-KD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

(…)bic-AraElectra-cross-encoder-NoKD-v1.tsv:   0%|          | 0.00/131M [00:00<?, ?B/s]

*****************************************
mmarco_run1000bm25Reranked/mmarco-Arabic-mMiniLML-cross-encoder-KD-v1.tsv
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.2356
recall_1000           	all	0.6369
*****************************************
mmarco_run1000bm25Reranked/mmarco-Arabic-mMiniLML-cross-encoder-NoKD-v1.tsv
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.2159
recall_1000           	all	0.6369
*****************************************
mmarco_run1000bm25Reranked/mmarco-Arabic-AraDPR-cross-encoder-KD-v1.tsv
*****************************************
msmarco run detected. Converting to trec...
recip_rank            	all	0.2423
recall_1000           	all	0.6369
*****************************************
mmarco_run1000bm25Reranked/mmarco-Arabic-AraDPR-cross-encoder-NoKD-v1.tsv
*****************************************
msmarco run detected. Converting